In [2]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class skyDataset(Dataset):
    def __init__(self, dataset_dim, data_path, label_path):
        super().__init__()
        self.image_w = 125
        self.data = np.memmap(data_path, dtype='float32', mode='r', shape=(dataset_dim, 3, self.image_w, self.image_w))
        self.label = np.memmap(label_path, dtype='uint8', mode='r', shape=(dataset_dim,))
        self.size = dataset_dim

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        image = self.data[idx]
        image = torch.from_numpy(np.array(image))

        label = torch.tensor(self.label[idx], dtype=torch.float32)

        return image, label

In [3]:
class skyModel(nn.Module):
    def __init__(self):
        super(skyModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 3, 5, 4)  # 5x5 con stride 5
        self.conv2 = nn.Conv2d(3, 3, 3, 2)  # 3x3 con stride 2
        self.relu = nn.ReLU()
        self.fc = nn.Linear(675, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x


def eval_acc(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y_true in data_loader:

            y_pred = model(x)
            y_pred = torch.squeeze(y_pred)
            y_pred = torch.round(y_pred)

            total += y_true.size(0)
            c = (y_pred == y_true).sum().item()
            correct += c

    return correct / total


In [4]:
train_data = os.path.join(".","dataset", "train_data.npm")
train_label = os.path.join(".","dataset", "train_label.npm")
test_data = os.path.join(".","dataset", "test_data.npm")
test_label = os.path.join(".","dataset", "test_label.npm")

train_dataset = skyDataset(7200, train_data, train_label)
test_dataset = skyDataset(2178, test_data, test_label)

dl_train = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
dl_test = DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0)

In [5]:
learning_rate = 0.01
epoch_num = 10

model = skyModel()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_values = []
train_accs = []
test_accs = []

for i in range(epoch_num):
    for j, (x, y_true) in enumerate(dl_train):
        y_pred = model(x)
        y_pred = torch.squeeze(y_pred)

        loss = loss_fn(y_pred, y_true)
        l = loss.item()
        loss_values.append(l)
        #print("Epoch:", i, "Batch:", j, "Loss:", l)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if j%100 == 0:
            train_acc = eval_acc(model, dl_train)
            test_acc = eval_acc(model, dl_test)
            print("Epoch:", i, "Batch:", j)
            print("Train acc:", train_acc)
            print("Test acc:", test_acc , "\n")
            train_accs.append(train_acc)
            test_accs.append(test_accs)


torch.save(model.state_dict(), "skyModel.bin")
#print(loss_values)
#print(train_accs)
#print(test_accs)

print("final train accuracy: ", eval_acc(model, dl_train))
print("final test accuracy: ", eval_acc(model, dl_test))

Epoch: 0 Batch: 0
Train acc: 0.49916666666666665
Test acc: 0.7396694214876033 

Epoch: 0 Batch: 100
Train acc: 0.9266666666666666
Test acc: 0.8650137741046832 

Epoch: 0 Batch: 200
Train acc: 0.9373611111111111
Test acc: 0.8865932047750229 

Epoch: 0 Batch: 300
Train acc: 0.9425
Test acc: 0.8714416896235078 

Epoch: 0 Batch: 400
Train acc: 0.9540277777777778
Test acc: 0.8801652892561983 

Epoch: 0 Batch: 500
Train acc: 0.9544444444444444
Test acc: 0.8948576675849403 

Epoch: 0 Batch: 600
Train acc: 0.9572222222222222
Test acc: 0.8806244260789715 

Epoch: 0 Batch: 700
Train acc: 0.9654166666666667
Test acc: 0.8966942148760331 

Epoch: 0 Batch: 800
Train acc: 0.9623611111111111
Test acc: 0.8769513314967861 

Epoch: 1 Batch: 0
Train acc: 0.9672222222222222
Test acc: 0.8925619834710744 

Epoch: 1 Batch: 100
Train acc: 0.9663888888888889
Test acc: 0.8875114784205693 

Epoch: 1 Batch: 200
Train acc: 0.9686111111111111
Test acc: 0.8847566574839302 

Epoch: 1 Batch: 300
Train acc: 0.9705555555